<a href="https://colab.research.google.com/github/loicpages/qmII_group_rep/blob/main/data_cleaning_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Data cleaning notebook - use to clean/ merge/ rearrange data sets***

*Libraries used:*

In [21]:
import pylab
import pandas as pd
import numpy as math
from google.colab import files
import io
import matplotlib.pyplot as plt
from functools import reduce
%matplotlib inline
plt.style.use('ggplot')

*Importing, Encording and Indexing data:*

In [3]:
data_path = '/content/complete_data_set.csv'
general_df = pd.read_csv(data_path, encoding ='latin-1')

In [150]:
#Creating a food_availibility dataframe#
intermediate_food_availibility_df = general_df[general_df.Commodity == "Total Grains/Cereals and Root Crops (R&T)"]
food_availibility_df = intermediate_food_availibility_df[intermediate_food_availibility_df.Item == 'Food Availability per capita']
food_availibility_df = food_availibility_df.reset_index(drop = True)
food_availibility_df.rename(columns = {'Amount': 'Food Availibility per capita (kg/cap/year)'}, inplace = True)
food_availibility_df.drop(['Commodity', 'Item', 'Unit'], axis=1, inplace = True)
#Creating a food_production dataframe#
intermediate_food_production_df = general_df[general_df.Commodity == "Total Grains/Cereals and Root Crops (R&T)"]
food_production_df = intermediate_food_production_df[intermediate_food_production_df.Item == 'Production Quantity']
food_production_df = food_production_df.reset_index(drop = True)
food_production_df.rename(columns = {'Amount': 'Production Quantity (Grains & Ce.+ Root Crops)(1000MT)'}, inplace = True)
food_production_df.drop(['Commodity', 'Item', 'Unit'], axis=1, inplace = True)
#Creating an import_grains_df#
intermediate_import_grains_df = general_df[general_df.Commodity == "Total Grains/Cereals"]
import_grains_df = intermediate_import_grains_df[intermediate_import_grains_df.Item == 'Import Quantity']
import_grains_df = import_grains_df.reset_index(drop = True)
import_grains_df.rename(columns = {'Amount': 'Import Quantity (Grains & Ce.)(1000MT)'}, inplace = True)
import_grains_df.drop(['Commodity', 'Item', 'Unit'], axis=1, inplace = True)
#Creating an export_grains_df#
intermediate_export_grains_df = general_df[general_df.Commodity == "Total Grains/Cereals"]
export_grains_df = intermediate_export_grains_df[intermediate_export_grains_df.Item == 'Export Quantity']
export_grains_df = export_grains_df.reset_index(drop = True)
export_grains_df.rename(columns = {'Amount': 'Export Quantity (Grains & Ce.)(1000MT)'}, inplace = True)
export_grains_df.drop(['Commodity', 'Item', 'Unit'], axis=1, inplace = True)
#Creating an import_root_df#
intermediate_import_root_df = general_df[general_df.Commodity == "Root Crops (R&T)"]
import_root_df = intermediate_import_root_df[intermediate_import_root_df.Item == 'Import Quantity']
import_root_df = import_root_df.reset_index(drop = True)
import_root_df.rename(columns = {'Amount': 'Import Quantity (Root Crops)(1000MT)'}, inplace = True)
import_root_df.drop(['Commodity', 'Item', 'Unit'], axis=1, inplace = True)
#Creating an export_root_df#
intermediate_export_root_df = general_df[general_df.Commodity == "Root Crops (R&T)"]
export_root_df = intermediate_export_root_df[intermediate_export_root_df.Item == 'Export Quantity']
export_root_df = export_root_df.reset_index(drop = True)
export_root_df.rename(columns = {'Amount': 'Export Quantity (Root Crops)(1000MT)'}, inplace = True)
export_root_df.drop(['Commodity', 'Item', 'Unit'], axis=1, inplace = True)
#Creating a population_df#
population_df = general_df[general_df.Commodity == "Population"]
population_df = population_df.reset_index(drop = True)
population_df.rename(columns = {'Amount': 'Population (Million)'}, inplace = True)
population_df.drop(['Commodity', 'Item', 'Unit'], axis=1, inplace = True)

In [ ]:
#merge data frames and fill gaps#
data_frames = [food_availibility_df, food_production_df, import_grains_df, export_grains_df, import_root_df, export_root_df, population_df]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Country', 'Year'],
                                            how='outer'), data_frames).fillna(pd.NaT)    

df_merged.sort_values(by=['Country', 'Year'], inplace = True)
df_merged.reset_index(drop = True)

In [166]:
#Add additional columns that are required for analysis#
df_merged['Imports (all)(1000MT)'] = df_merged['Import Quantity (Grains & Ce.)(1000MT)'] + df_merged['Import Quantity (Root Crops)(1000MT)']
df_merged['Exports (all)(1000MT)'] = df_merged['Export Quantity (Grains & Ce.)(1000MT)'] + df_merged['Export Quantity (Root Crops)(1000MT)']
df_merged['Production Quantity per capita (kg/cap/year)'] = df_merged['Production Quantity (Grains & Ce.+ Root Crops)(1000MT)'] / df_merged['Population (Million)']
df_merged['Imports (all) per capita (kg/cap/year)'] = df_merged['Imports (all)(1000MT)'] / df_merged['Population (Million)']
df_merged['Exports (all) per capita (kg/cap/year)'] = df_merged['Exports (all)(1000MT)'] / df_merged['Population (Million)']
df_merged['Production + net ex-imports (kg/cap/year)'] = (df_merged['Imports (all) per capita (kg/cap/year)'] + df_merged['Production Quantity per capita (kg/cap/year)']) - df_merged['Exports (all) per capita (kg/cap/year)']

#save as csv file#
df_merged.to_csv(r'df_final.csv', index = False)

#Save as csv without missing values#
df_no_missing_values = df_merged.dropna()
df_no_missing_values.to_csv(r'df_final(no_missing values).csv', index = False)
